In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img1 = cv2.imread('../data/GEarth/left.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('../data/GEarth/front_left.png', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('../data/GEarth/front.png', cv2.IMREAD_GRAYSCALE)

images = [img1, img2, img3]


In [20]:
def stitch_images(images):
    # Check and convert images
    processed_images = []
    for img in images:
        if img is None:
            print("One of the input images is None.")
            return None
        
        # Ensure the image is in BGR format
        if len(img.shape) == 2:  # Image is grayscale
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        elif img.shape[2] == 4:  # Image is RGBA
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
        
        processed_images.append(img)

    # Create a Stitcher instance
    stitcher = cv2.Stitcher.create(cv2.Stitcher_PANORAMA)
    
    # Perform the stitching process
    status, stitched_image = stitcher.stitch(processed_images)
    
    if status == cv2.Stitcher_OK:
        print("Images stitched successfully.")
        # Display the stitched image
        return stitched_image
    
res_stitch = stitch_images(images)
cv2.imwrite('../results/pano.png', res_stitch)

Images stitched successfully.


True

In [21]:
points2 = []

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Append the new point (x, y)
        points2.append((x, y))
        
        # Draw a small circle at the click point
        cv2.circle(img, (x, y), 3, (255, 255, 255), -1)
        
        # Draw the polygon if there are at least two points
        if len(points2) >= 2:
            cv2.polylines(img, [np.array(points2)], isClosed=False, color=(255, 255, 255), thickness=2)
        
        cv2.imshow('image', img)

# Load an image
img = res_stitch
if img is None:
    print("Error: Image not found.")
else:
    # Create a window
    cv2.namedWindow('image')
    cv2.setMouseCallback('image', click_event)

    # Display the image
    cv2.imshow('image', img)

    # Wait until any key is pressed
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [22]:
height, width = res_stitch.shape[:2]

src_pts1 = np.array(points2, dtype=np.float32)
dst_pts = np.array([[0, 0], [0, height], [width, height], [width, 0]], dtype=np.float32)

M1 = cv2.getPerspectiveTransform(src_pts1, dst_pts)
warped_image1 = cv2.warpPerspective(res_stitch, M1, (width, height))

cv2.imwrite('../results/warped_pano.png', warped_image1)

True